<a href="https://colab.research.google.com/github/SangHee33/Sanghee2/blob/main/%ED%95%B8%EC%A6%88%EC%98%A8_9%EC%9E%A5_%EB%B9%84%EC%A7%80%EB%8F%84_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

p. 299 ~ 345

#9. 비지도 학습
- 군집 : 비슷한 샘플을 클러스터로 모음. 군집은 데이터 분석, 고객 분류, 추천 시스템, 검색 엔진, 이미지 분할, 준지도 학습, 차원 축소 등에 사용 가능
- 이상치 탐지 : '정상' 데이터가 어떻게 보이는지를 학습. 비정상 샘플을 감지하는 데 사용
- 밀도 추정 : 데이터셋 생성 확률 과정의 확률 밀도 함수를 추정. 밀도 추정은 이상치 탐지에 널리 사용. 밀도가 매우 낮은 영역에 놓인 샘플이 이상치일 가능성이 높음. 데이터 분석과 시각화에도 유용

##9.1 군집
- 비슷한 샘플을 구별해 하나의 클러스터 또는 비슷한 샘플의 그룹으로 할당
- 사용 분야 : 고객 분류, 데이터 분석, 차원 축소 기법, 이상치 탐지, 준지도 학습, 검색 엔진, 이미지 분할
- 센트로이드라 부르는 특정 포인트를 중심으로 모인 샘플을 찾거나 샘플이 밀집되어 연속된 영역을 찾기도

###9.1.1 k-평균
- 각 클러스터의 중심을 찾고 가장 가까운 클러스터에 샘플을 할당

In [26]:
from sklearn.datasets import make_blobs
import numpy as np


blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])
X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

In [35]:
from sklearn.cluster import KMeans
k = 5
kmeans = KMeans(n_clusters=k)
y_pred = kmeans.fit_predict(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


- 군집에서 각 샘플의 레이블은 알고리즘이 샘플에 할당한 클러스터의 인덱스

In [28]:
y_pred

array([4, 1, 0, ..., 3, 0, 1], dtype=int32)

In [29]:
y_pred is kmeans.labels_

True

In [30]:
# 센트로이트
kmeans.cluster_centers_

array([[ 0.20876306,  2.25551336],
       [-2.80389616,  1.80117999],
       [-1.46679593,  2.28585348],
       [-2.79290307,  2.79641063],
       [-2.80037642,  1.30082566]])

In [31]:
X_new = np.array([[0,2],[3,2],[-3,3],[-3,2.5]])
kmeans.predict(X_new)

array([0, 0, 3, 3], dtype=int32)

- 클러스터의 크기가 많이 다르면 잘 작동하지 않음
- 하드 군집 : 샘플을 하나의 클러스터에 할당
- 소프트 군집 : 클러스터마다 샘플에 점수를 부여

In [32]:
# 샘플과 센트로이드 사이의 거리 반환
kmeans.transform(X_new)

array([[0.32995317, 2.81093633, 1.49439034, 2.9042344 , 2.88633901],
       [2.80290755, 5.80730058, 4.4759332 , 5.84739223, 5.84236351],
       [3.29399768, 1.21475352, 1.69136631, 0.29040966, 1.71086031],
       [3.21806371, 0.72581411, 1.54808703, 0.36159148, 1.21567622]])

**센트로이드 초기화 방법**

In [33]:
good_init = np.array([[-3,3],[-3,2],[-3,1],[-1,2],[0,2]])
kmeans = KMeans(n_clusters=5, init=good_init, n_init=1)

- n_init : 랜덤 초기화 횟수
- 이니셔 : 각 샘플과 가장 가까운 센트로이드 사이의 평균 제곱 거리
- 이니셔가 낮은 모델 반환

In [37]:
kmeans.inertia_

211.5985372581684

In [36]:
kmeans.score(X)

-211.59853725816836

**k-평균 속도 개선과 미니배치 k-평균**
- 전체 데이터셋을 사용해 반복하지 않고 반복마다 미니배치를 사용해 센트로이드를 조금씩 이동
- 알고리즘의 속도를 3배에서 4배 정도 높임
- MiniBatchKMeans 클래스

In [38]:
from sklearn.cluster import MiniBatchKMeans

minibatch_kmeans = MiniBatchKMeans(n_clusters=5)
minibatch_kmeans.fit(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


MiniBatchKMeans(n_clusters=5)

- 미니배치 k-평균 알고리즘이 일반 k-평균 알고리즘보다 훨씬 빠르지만 이니셔는 일반적으로 조금 더 나쁨

**최적의 클러스터 개수 찾기**
- 이니셔는 k가 증가할수록 감소
- 이니셔가 가장 크게 감소하는 지점
- 실루엣 점수 : +1에 가까우면 클러스터 안에 잘 속해 있고, 0은 클러스터 경계에 위치하고, -1은 잘못된 클러스터에 할당됨

In [39]:
from sklearn.metrics import silhouette_score
silhouette_score(X,kmeans.labels_)

0.655517642572828

###9.1.2 k-평균의 한계
- 여러 번 실행해야하고 클러스터 개수를 지정해야함
- 클러스터의 크기나 밀집도가 서로 다르거나 원형이 아닐 경우 잘 작동하지 않음

###9.1.3 군집을 사용한 이미지 분할
- 이미지를 세그먼트 여러 개로 분할하는 작업
- 시맨틱 분할 : 동일한 종류의 물체에 속한 모든 픽셀은 같은 세그먼트
- 색상 분할 : 동일한 색상을 가진 픽셀을 같은 세그먼트에 할당



In [41]:
import os
import urllib.request

PROJECT_ROOT_DIR = "."
CHAPTER_ID = "unsupervised_learning"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)


images_path = os.path.join(PROJECT_ROOT_DIR, "images", "unsupervised_learning")
os.makedirs(images_path, exist_ok=True)
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/rickiepark/handson-ml2/master/"
filename = "ladybug.png"
print("Downloading", filename)
url = DOWNLOAD_ROOT + "images/unsupervised_learning/" + filename
urllib.request.urlretrieve(url, os.path.join(images_path, filename))

('./images/unsupervised_learning/ladybug.png',
 <http.client.HTTPMessage at 0x7d6c5c674550>)

In [42]:
from matplotlib.image import imread
image = imread(os.path.join("images","unsupervised_learning","ladybug.png"))
image.shape

(533, 800, 3)

In [43]:
X = image.reshape(-1, 3)
kmeans = KMeans(n_clusters=8, random_state=42).fit(X)
segmented_img = kmeans.cluster_centers_[kmeans.labels_]
segmented_img = segmented_img.reshape(image.shape)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


###9.1.4 군집을 사용한 전처리
- 차원 축소에 효과적
- 지도 학습 알고리즘을 적용하기 전에 전처리 단계로 사용

In [54]:
from sklearn.datasets import load_digits

X_digits, y_digits = load_digits(return_X_y=True)

In [55]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X_digits,y_digits)

In [56]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [57]:
log_reg.score(X_test,y_test)

0.9733333333333334

In [58]:
# k-평균을 전처리 단계로 사용
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ("kmeans",KMeans(n_clusters=50)),
    ("log_reg",LogisticRegression())
])
pipeline.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                ('log_reg', LogisticRegression())])

In [59]:
pipeline.score(X_test,y_test)

0.9666666666666667

In [60]:
# 더 좋은 k값 찾기
from sklearn.model_selection import GridSearchCV

param_grid = dict(kmeans__n_clusters=range(2,100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train,y_train)

Fitting 3 folds for each of 98 candidates, totalling 294 fits


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[CV] END ...............................kmeans__n_clusters=2; total time=   0.4s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[CV] END ...............................kmeans__n_clusters=2; total time=   0.9s


...

[CV] END ..............................kmeans__n_clusters=99; total time=   2.6s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[CV] END ..............................kmeans__n_clusters=99; total time=   2.3s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                                       ('log_reg', LogisticRegression())]),
             param_grid={'kmeans__n_clusters': range(2, 100)}, verbose=2)

In [61]:
grid_clf.best_params_

{'kmeans__n_clusters': 68}

In [62]:
grid_clf.score(X_test,y_test)

0.9644444444444444

###9.1.5 군집을 사용한 준지도 학습
- 레이블이 없는 데이터가 많고 레이블이 있는 데이터는 적을 때 사용

In [63]:
n_labeled = 50
log_reg = LogisticRegression()
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [64]:
log_reg.score(X_test,y_test)

0.8844444444444445

- 저네 데이터셋을 사용했을 때보다 낮은 정확도
- 대표 이미지 : 훈련 세트를 클러스터로 모았을 때, 각 클러스터에서 센트로이드에 가장 가까운 이미지

In [66]:
k =50
kmeans = KMeans(n_clusters=k)
X_digits_dist = kmeans.fit_transform(X_train)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train[representative_digit_idx]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [67]:
y_train[representative_digit_idx]

array([3, 1, 0, 6, 4, 5, 7, 8, 9, 1, 2, 5, 9, 6, 6, 2, 8, 4, 7, 0, 1, 5,
       2, 4, 3, 7, 6, 9, 8, 3, 9, 3, 3, 6, 4, 1, 1, 8, 7, 8, 2, 4, 9, 0,
       2, 1, 2, 5, 7, 5])

In [68]:
y_representative_digits = np.array([3, 1, 0, 6, 4, 5, 7, 8, 9, 1, 2, 5, 9, 6, 6, 2, 8, 4, 7, 0, 1, 5,
       2, 4, 3, 7, 6, 9, 8, 3, 9, 3, 3, 6, 4, 1, 1, 8, 7, 8, 2, 4, 9, 0,
       2, 1, 2, 5, 7, 5])

In [69]:
log_reg = LogisticRegression()
log_reg.fit(X_representative_digits, y_representative_digits)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9111111111111111

In [70]:
# 레이블 전파
y_train_propagated = np.empty(len(X_train), dtype=np.int32)
for i in range(k):
    y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]

In [71]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train_propagated)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9177777777777778

In [73]:
percentile_closest = 20

X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
    in_cluster = (kmeans.labels_ == i)
    cluster_dist = X_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest)
    above_cutoff = (X_cluster_dist > cutoff_distance)
    X_cluster_dist[in_cluster & above_cutoff] = -1

partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

In [74]:
log_reg = LogisticRegression()
log_reg.fit(X_train_partially_propagated, y_train_partially_propagated)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.92

In [75]:
np.mean(y_train_partially_propagated == y_train[partially_propagated])

0.9897610921501706

### 9.1.6 DBSCAN
- 밀집된 연속적 지역을 클러스터로 정의
- 모든 클러스터가 충분히 밀집되어 있고 밀집되지 않은 지역과 잘 구분될 때 좋은 성능

In [76]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X, y=make_moons(n_samples=1000, noise=0.05)
dbscan=DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

DBSCAN(eps=0.05)

In [77]:
dbscan.labels_

array([ 0,  1,  2,  3,  3,  4,  3,  1,  5,  0,  0,  7,  6,  5,  4,  7,  5,
        0,  3,  1,  2,  7,  2,  2,  4,  4,  0,  2,  4,  6,  2,  1,  5,  4,
        1,  7,  8,  5,  1,  1, -1,  3, -1,  2,  7,  1,  2,  3, -1,  1,  1,
        1,  4,  3,  5,  7,  7,  3,  5,  2,  7,  3,  5,  6,  9,  6,  3,  0,
        6,  1,  6,  6,  2,  1,  7,  2,  2,  2,  7,  2,  1,  1,  5,  7,  7,
       -1, 11,  2,  3,  5,  1,  1,  1,  1,  2,  1,  2,  7,  1,  7,  2,  5,
        1,  7,  7,  3,  3, 10,  2, -1,  1,  9,  1,  1,  1,  2,  7,  3,  0,
       -1,  2,  1,  8,  3,  1, -1,  6,  1,  3,  1,  0, -1,  2,  1,  3,  1,
        3,  0,  5,  3,  3,  1,  2,  6,  1,  4,  1,  2,  3,  3,  2,  2,  3,
        1,  2,  5,  2,  3,  3,  1,  1,  3,  1,  7, -1,  6,  0, -1,  3,  2,
        1,  3,  3,  2,  4,  0, -1, -1,  2,  2,  2,  3,  0, 10,  6,  1,  7,
        3,  8,  6,  2,  7,  2,  2,  2,  3,  3,  3,  1,  3,  2,  1,  1,  9,
        7,  0,  3,  5,  4,  4,  7,  1,  1,  9,  3,  2,  0,  4,  1,  7,  1,
        4,  2,  7,  1, -1

In [78]:
len(dbscan.core_sample_indices_)

796

In [79]:
# 핵심 샘플의 인덱스
dbscan.core_sample_indices_

array([  0,   1,   2,   3,   4,   5,   6,   8,   9,  10,  12,  14,  16,
        17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  28,  29,  30,
        31,  33,  34,  35,  36,  37,  38,  39,  41,  43,  44,  45,  46,
        50,  51,  52,  53,  54,  56,  57,  58,  59,  60,  61,  64,  65,
        66,  67,  68,  69,  71,  72,  73,  74,  75,  76,  77,  78,  80,
        81,  82,  83,  88,  89,  90,  91,  92,  93,  96,  97,  98,  99,
       100, 101, 102, 104, 106, 107, 108, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 120, 121, 123, 124, 126, 127, 128, 129, 130, 133,
       134, 135, 136, 137, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 159, 160, 162, 165, 166,
       168, 169, 170, 171, 172, 173, 174, 175, 178, 179, 180, 181, 185,
       186, 187, 189, 190, 192, 193, 194, 195, 196, 197, 198, 199, 200,
       201, 202, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214,
       215, 216, 217, 218, 221, 222, 223, 224, 227, 229, 230, 23

In [80]:
dbscan.components_

array([[ 1.0295325 , -0.55717145],
       [ 1.88368755, -0.0496506 ],
       [-0.04033148,  0.9926744 ],
       ...,
       [-0.74870848,  0.5786591 ],
       [ 0.10978988, -0.0357529 ],
       [-0.37777068,  0.90444689]])

In [81]:
from sklearn.neighbors import KNeighborsClassifier

knn=KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])

KNeighborsClassifier(n_neighbors=50)

In [84]:
X_new=np.array([[-0.5, 0], [0, 0.5], [1, -0.1], [2,1]])
knn.predict(X_new)

array([5, 3, 7, 1])

In [83]:
y_dist, y_pred_idx=knn.kneighbors(X_new, n_neighbors=1)
y_pred=dbscan.labels_[dbscan.core_sample_indices_][y_pred_idx]
y_pred[y_dist>0.2]=-1
y_pred.ravel()

array([-1,  9,  7, -1])

- 클러스터의 모양과 개수에 상관없이 감지할 수 있는 능력
- 이상치에 안정적이고 하이퍼파라미터가 두 개
- 클러스터 간의 밀집도가 크게 다르면 모든 클러스터를 올바르게 잡아내는 것이 불가능

###9.1.7 다른 군집 알고리즘
- 병합 군집 : 클러스터 계층을 밑바닥부터 위로 쌓아 구성
- BIRCH : 대규모 데이터셋, 특성 개수가 너무 많지 않다면 배치 k-평균보다 빠르고 비슷한 겨롸
- 평균-이동 : 각 샘플을 중심으로 하는 원을 그리고 원마다 안에 포함된 모든 샘플의 평균을 구해 원의 중심을 평균점으로 이동
- 유사도 전파 : 샘플은 자신을 대표할 수 있는 비슷한 샘플에 투표하고 알고리즘이 수렴하면 각 대표와 투표한 샘플이  클러스터 형성
- 스펙트럼 군집 : 샘플 사이의 유사도 행렬을 받아 저차원 임베딩 만들고 저차원 공간에서 또 다른 군집 알고리즘을 사용

##9.2 가우시안 혼합
- 샘플이 파라미터가 알려지지 않은 여러 개의 혼합된 가우시안 분포에서 생성되었다고 가정하는 확률 모델
- 하나의 가우시안 분포에서 생성된 모든 샘플은 하나의 클러스터를 형성
- 일반적으로 타원형
- 각 클러스터는 타원의 모양, 크기, 밀집도, 방향이 다름

In [86]:
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=3, n_init=10)
gm.fit(X)

GaussianMixture(n_components=3, n_init=10)

In [87]:
gm.weights_

array([0.20616603, 0.60102595, 0.19280802])

In [88]:
gm.means_

array([[ 1.7336904 , -0.06332235],
       [ 0.48273153,  0.26669014],
       [-0.76514103,  0.52942515]])

In [89]:
gm.covariances_

array([[[ 0.05371716,  0.06355943],
        [ 0.06355943,  0.09124721]],

       [[ 0.17653273, -0.11005255],
        [-0.11005255,  0.28869186]],

       [[ 0.04288512,  0.0543556 ],
        [ 0.0543556 ,  0.08380618]]])

- 기댓값-최대화 알고리즘 : 클러스터 파라미터를 랜덤하게 초기화하고 수렴할 때까지 두 단계를 반복, 먼저 샘플을 클러스터에 할당(기댓값 단계), 클러스터를 업데이트(최대화 단계)

In [92]:
gm.converged_

True

In [93]:
gm.n_iter_

16

In [94]:
gm.predict(X)

array([1, 0, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 0, 1, 1,
       2, 2, 1, 1, 1, 1, 1, 1, 2, 0, 2, 1, 0, 1, 1, 2, 0, 0, 0, 1, 1, 2,
       1, 0, 2, 1, 0, 0, 0, 0, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 2, 1, 1, 2, 0, 0, 2, 1, 1, 1, 1, 1,
       1, 2, 0, 0, 0, 0, 2, 0, 1, 1, 0, 1, 2, 2, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 2,
       1, 0, 1, 0, 1, 1, 2, 1, 1, 0, 1, 1, 0, 1, 0, 2, 1, 1, 1, 2, 1, 0,
       2, 2, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 2, 1, 2, 0, 1, 1, 1, 1, 1,
       2, 1, 2, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 2, 1, 0, 2, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 2,
       1, 1, 0, 2, 1, 2, 0, 0, 1, 1, 0, 0, 0, 2, 1, 1, 0, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1,
       1, 1, 1, 0, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,

In [95]:
gm.predict_proba(X)

array([[2.49017126e-003, 9.97509829e-001, 1.45439776e-180],
       [9.97653436e-001, 2.34656447e-003, 2.11192232e-261],
       [2.46183664e-147, 9.99983709e-001, 1.62912962e-005],
       ...,
       [4.55150579e-199, 3.49411785e-003, 9.96505882e-001],
       [5.46157370e-062, 1.00000000e+000, 1.45520902e-044],
       [1.32297851e-180, 3.02238097e-001, 6.97761903e-001]])

In [96]:
X_new, y_new = gm.sample(6)
X_new

array([[ 0.10794085,  0.11086631],
       [ 0.40695992, -0.02725594],
       [ 0.63490863,  0.07341996],
       [ 0.46386823,  0.07618625],
       [ 0.4953513 ,  0.07395223],
       [-1.00044196,  0.2981554 ]])

In [97]:
y_new

array([1, 1, 1, 1, 1, 2])

In [98]:
# 확률 밀도 함수의 로그를 예측
gm.score_samples(X)

array([-2.09730230e+00, -9.32078774e-01, -1.86121020e+00, -1.57974061e+00,
       -1.82235686e+00, -1.68825030e+00, -1.47105768e+00, -9.57082750e-01,
       -3.02274673e-01, -1.61586461e+00, -1.70832984e+00, -2.01146215e+00,
       -1.59472213e+00, -1.44798822e+00, -1.31807144e+00, -1.32905483e+00,
       -1.94825728e-01, -1.83409465e+00, -1.65049227e+00, -2.65895971e-01,
       -1.74010490e+00, -1.53871127e+00, -6.86978115e-01, -5.95695736e-01,
       -1.87072569e+00, -1.59766006e+00, -2.12707032e+00, -1.95899095e+00,
       -1.52908964e+00, -1.38347465e+00, -7.97191817e-01, -6.64944671e-01,
       -3.70878699e+00, -1.30251979e+00,  1.18203372e-02, -1.79185969e+00,
       -1.51423946e+00, -2.30643543e-01, -9.01182091e-01, -1.31978006e+00,
       -3.19758710e+00, -1.62081012e+00, -2.06718447e+00,  1.88623766e-01,
       -1.62227194e+00, -6.88016045e-01,  1.95331030e-01, -1.25854138e+00,
        2.91009706e-02, -2.86193916e+00, -1.13283701e+00, -2.88305684e-01,
       -1.52227574e+00, -

###9.2.1 가우시안 혼합을 사용한 이상치 탐지
- 밀도가 낮은 지역에 있는 모든 샘플을 이상치로 볼 수 있음
- 밀도 임곗값을 정해야 함

In [99]:
densities=gm.score_samples(X)
density_threshold=np.percentile(densities, 4)
anomalies=X[densities < density_threshold]

###9.2.2 클러스터 개수 선택하기
- BIC나 AIC와 같은 이론적 정보 기준을 최소화하는 모델 찾기

In [100]:
gm.bic(X)

2738.265403743947

In [101]:
gm.aic(X)

2654.8335640012506

###9.2.3 베이즈 가우시안 혼합 모델
- 불필요한 클러스터의 가중치를 0으로 만듦

In [102]:
from sklearn.mixture import BayesianGaussianMixture
bgm=BayesianGaussianMixture(n_components=10, n_init=10)
bgm.fit(X)
np.round(bgm.weights_, 2)

array([0.19, 0.16, 0.12, 0.12, 0.15, 0.14, 0.13, 0.  , 0.  , 0.  ])

###9.2.4 이상치 탐지와 특이치 탐지를 위한 다른 알고리즘
- PCA
- Fast-MCD
- 아이솔레이션 포레스트
- LOF
- one-class SVM